In [99]:
import pandas as pd
import numpy as np
import os
import FinanceDataReader as fdr
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from tqdm import tqdm
from sklearn.model_selection import GridSearchCV

from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt
from statsmodels.tsa.arima.model import ARIMA
from pandas import datetime
from pmdarima.arima import auto_arima

<ipython-input-99-6ad24988a1b9>:13: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  from pandas import datetime


In [100]:
path = './open'
list_name = 'Stock_List.csv'
sample_name = 'sample_submission.csv'

stock_list = pd.read_csv(os.path.join(path,list_name))
stock_list['종목코드'] = stock_list['종목코드'].apply(lambda x : str(x).zfill(6))
sample_stock_list=stock_list[0:5]

In [101]:
# select start, end data

start_date = '20200613'
end_date = '20211031'

start_weekday = pd.to_datetime(start_date).weekday()
max_weeknum = pd.to_datetime(end_date).strftime('%V')
Business_days = pd.DataFrame(pd.date_range(start_date,end_date,freq='B'), columns = ['Date'])

print(f'WEEKDAY of "start_date" : {start_weekday}')
print(f'NUM of WEEKS to "end_date" : {max_weeknum}')
print(f'HOW MANY "Business_days" : {Business_days.shape}', )
display(Business_days.head())

WEEKDAY of "start_date" : 5
NUM of WEEKS to "end_date" : 43
HOW MANY "Business_days" : (360, 1)


,Date
0,2020-06-15
1,2020-06-16
2,2020-06-17
3,2020-06-18
4,2020-06-19


In [102]:
sample_code = stock_list.loc[0,'종목코드']

sample = fdr.DataReader(sample_code, start = start_date, end = end_date)[['Close']].reset_index()
sample = pd.merge(Business_days, sample, how = 'outer')
sample['weekday'] = sample.Date.apply(lambda x : x.weekday())
sample['weeknum'] = sample.Date.apply(lambda x : x.strftime('%V'))
sample.Close = sample.Close.ffill()
sample = pd.pivot_table(data = sample, values = 'Close', columns = 'weekday', index = 'weeknum')
sample.head()

weekday,0,1,2,3,4
weeknum,,,,,
01,83000.0,83900.0,82200.0,82900.0,88800.0
02,91000.0,90600.0,89700.0,89700.0,88000.0
03,85000.0,87000.0,87200.0,88100.0,86800.0
04,89400.0,86700.0,85600.0,83700.0,82000.0
05,83000.0,84400.0,84600.0,82500.0,83500.0


In [103]:
# training, predict method
def get_prediction(x_close, y_close, x_close_public):
    
    prediction_close = 0.0
    for i, model in enumerate(models):         
        model.fit(x_close, y_close)
        prediction_close += model.predict(np.expand_dims(x_close_public, 0))*models_rate[i]
    
    return prediction_close

sample_name = 'sample_submission.csv'
sample_submission = pd.read_csv(os.path.join(path,sample_name))

In [104]:
model1 = LinearRegression()
# model2 = RandomForestRegressor(criterion="mae", n_estimators=200)
models = [model1]


model1_rate = 1
# model2_rate = 0.3
models_rate = [model1_rate]
pred_case=[]
for code in tqdm(sample_stock_list['종목코드'].values):
    data = fdr.DataReader(code, start = start_date, end = end_date)[['Close']].reset_index()
    data = pd.merge(Business_days, data, how = 'outer')
    data['weekday'] = data.Date.apply(lambda x : x.weekday())
    data['weeknum'] = data.Date.apply(lambda x : x.strftime('%V'))
    data.Close = data.Close.ffill()
    data.Close = data.Close.bfill()

    data['ma5']=data['Close'].ewm(5).mean()
    data['ma20']=data['Close'].ewm(20).mean()




    data_close = pd.pivot_table(data = data, values = 'Close', columns = 'weekday', index = 'weeknum')
    data_ma5 = pd.pivot_table(data = data, values = 'ma5', columns = 'weekday', index = 'weeknum')
    data_ma20 = pd.pivot_table(data = data, values = 'ma20', columns = 'weekday', index = 'weeknum')

    # data flatten  # data flatten for reshape
    data_close_flatten = np.ravel(data_close.to_numpy())
    data_ma5_flatten = np.ravel(data_ma5.to_numpy())
    data_ma20_flatten = np.ravel(data_ma20.to_numpy())
    predictions = []
    predictions_ma5 = []
    predictions_ma20 = []

    
    # predict day 1

    # close
    data_close_day1 = data_close_flatten.reshape((-1,1))
    data_ma5_day1 = data_ma5_flatten.reshape((-1,1))
    data_ma20_day1 = data_ma20_flatten.reshape((-1,1))

    x_close = data_close_day1[:-2]  # training x
    y_close = data_close_day1[1:-1]  # training y
    x_close_public = data_close_day1[-1]  # predict x

    x_ma5 = data_ma5_day1[:-2]  # training x
    y_ma5 = data_ma5_day1[1:-1]  # training y
    x_ma5_public = data_ma5_day1[-1]  # predict x

    x_ma20 = data_ma20_day1[:-2]  # training x
    y_ma20 = data_ma20_day1[1:-1]  # training y
    x_ma20_public = data_ma20_day1[-1]  # predict x

    prediction_close = get_prediction(x_close, y_close, x_close_public)
    prediction_ma5 = get_prediction(x_ma5, y_ma5, x_ma5_public)
    prediction_ma20 = get_prediction(x_ma20, y_ma20, x_ma20_public)
    predictions.append(prediction_close*0.7+prediction_ma5*0.25+prediction_ma20*0.05)
    
    # predict day 1 finish
    
    
    # predict day 2
    
    # close

    data_close_day2 = data_close_flatten.reshape((-1,2))
    data_ma5_day2 = data_ma5_flatten.reshape((-1,2))
    data_ma20_day2 = data_ma20_flatten.reshape((-1,2))


    x_close = data_close_day2[:-2]
    y_close = data_close_day2[1:-1][:, 1]  # 2일차만 훈련, 예측함
    x_close_public = data_close_day2[-1]   

    x_ma5 = data_ma5_day2[:-2]  # training x
    y_ma5 = data_ma5_day2[1:-1][:,1]  # training y
    x_ma5_public = data_ma5_day2[-1]  # predict x

    x_ma20 = data_ma20_day2[:-2]  # training x
    y_ma20 = data_ma20_day2[1:-1][:,1]  # training y
    x_ma20_public = data_ma20_day2[-1]  # predict x

    prediction_close = get_prediction(x_close, y_close, x_close_public)
    prediction_ma5 = get_prediction(x_ma5, y_ma5, x_ma5_public)
    prediction_ma20 = get_prediction(x_ma20, y_ma20, x_ma20_public)
    predictions.append(prediction_close*0.7+prediction_ma5*0.25+prediction_ma20*0.05)
    
    # predict day 2 finish
    
    
    # predict day 3
    
    # close
    # data_close_day3 = np.insert(data_close_day3, 0, data_close_day3[0])
    # data_close_day3 = data_close_day3.reshape((-1, 3))

    # data_ma5_day3 = np.insert(data_ma5_day3, 0, data_ma5_day3[0])
    # data_ma5_day3 = data_ma5_day3.reshape((-1, 3))

    # data_ma20_day3 = np.insert(data_ma20_day3, 0, data_ma20_day3[0])
    # data_ma20_day3 = data_ma20_day3.reshape((-1, 3))

    data_close_day3 = data_close_flatten.reshape((-1,3))
    data_ma5_day3 = data_ma5_flatten.reshape((-1,3))
    data_ma20_day3 = data_ma20_flatten.reshape((-1,3))

    x_close = data_close_day3[:-2]
    y_close = data_close_day3[1:-1][:, 2]
    x_close_public = data_close_day3[-1]

    x_ma5 = data_ma5_day3[:-2]  # training x
    y_ma5 = data_ma5_day3[1:-1][:,2]  # training y
    x_ma5_public = data_ma5_day3[-1]  # predict x

    x_ma20 = data_ma20_day3[:-2]  # training x
    y_ma20 = data_ma20_day3[1:-1][:,2]  # training y
    x_ma20_public = data_ma20_day3[-1]  # predict x

    prediction_close = get_prediction(x_close, y_close, x_close_public)
    prediction_ma5 = get_prediction(x_ma5, y_ma5, x_ma5_public)
    prediction_ma20 = get_prediction(x_ma20, y_ma20, x_ma20_public)
    predictions.append(prediction_close*0.7+prediction_ma5*0.25+prediction_ma20*0.05)
    # predict day 3 finish

    
    # predict day 4
    
    # close
    # data_close_day4 = np.insert(data_close_flatten, 0, data_close_flatten[0])
    # data_close_day4 = np.insert(data_close_day4, 0, data_close_day4[0])
    # data_close_day4 = np.append(data_close_day4, 0)
    # data_close_day4 = np.append(data_close_day4, 0)
    # data_close_day4 = np.append(data_close_day4, 0)
    data_close_day4 = data_close_flatten.reshape((-1, 4))

    # data_ma5_day4 = np.insert(data_ma5_flatten, 0, data_ma5_flatten[0])
    # data_ma5_day4 = np.insert(data_ma5_day4, 0, data_ma5_day4[0])
    # data_ma5_day4 = np.append(data_ma5_day4, 0)
    # data_ma5_day4 = np.append(data_ma5_day4, 0)
    # data_ma5_day4 = np.append(data_ma5_day4, 0)
    data_ma5_day4 = data_ma5_flatten.reshape((-1, 4))

    # data_ma20_day4 = np.insert(data_ma20_flatten, 0, data_ma20_flatten[0])
    # data_ma20_day4 = np.insert(data_ma20_day4, 0, data_ma20_day4[0])
    # data_ma20_day4 = np.append(data_ma20_day4, 0)
    # data_ma20_day4 = np.append(data_ma20_day4, 0)
    # data_ma20_day4 = np.append(data_ma20_day4, 0)
    data_ma20_day4 = data_ma20_flatten.reshape((-1, 4))

    x_close = data_close_day4[:-2]
    y_close = data_close_day4[1:-1][:, 3]
    x_close_public = data_close_day4[-1]

    x_ma5 = data_ma5_day4[:-2]  # training x
    y_ma5 = data_ma5_day4[1:-1][:,3]  # training y
    x_ma5_public = data_ma5_day4[-1]  # predict x

    x_ma20 = data_ma20_day4[:-2]  # training x
    y_ma20 = data_ma20_day4[1:-1][:,3]  # training y
    x_ma20_public = data_ma20_day4[-1]  # predict x

    prediction_close = get_prediction(x_close, y_close, x_close_public)
    prediction_ma5 = get_prediction(x_ma5, y_ma5, x_ma5_public)
    prediction_ma20 = get_prediction(x_ma20, y_ma20, x_ma20_public)
    predictions.append(prediction_close*0.7+prediction_ma5*0.25+prediction_ma20*0.05)
    
    # predict day 4 finish
    
    # predict day 5
    
    # close
    data_close_day5 = data_close_flatten.reshape((-1, 5))
    data_ma5_day5 = data_ma5_flatten.reshape((-1, 5))
    data_ma20_day5 = data_ma20_flatten.reshape((-1, 5))
    
    x_close = data_close_day5[:-2]
    y_close = data_close_day5[1:-1][:, 4]
    x_close_public = data_close_day5[-1]

    x_ma5 = data_ma5_day5[:-2]  # training x
    y_ma5 = data_ma5_day5[1:-1][:,4]  # training y
    x_ma5_public = data_ma5_day5[-1]  # predict x

    x_ma20 = data_ma20_day5[:-2]  # training x
    y_ma20 = data_ma20_day5[1:-1][:,4]  # training y
    x_ma20_public = data_ma20_day5[-1]  # predict x

    prediction_close = get_prediction(x_close, y_close, x_close_public)
    prediction_ma5 = get_prediction(x_ma5, y_ma5, x_ma5_public)
    prediction_ma20 = get_prediction(x_ma20, y_ma20, x_ma20_public)
    predictions.append(prediction_close*0.7+prediction_ma5*0.25+prediction_ma20*0.05)
    # predict day 5 finish

        # autoarima
    arima_data = data[:-1]
    model = auto_arima(arima_data.Close,
    start_p=1, start_q=1,d=1,
    max_p=3, max_q=3,max_d=2,
    m=7,
    start_P=1,start_Q=1,D=1,
    max_P=2,max_Q=2,max_D=2,
                           seasonal=True,
                           trace=True,
                           error_action='ignore',  
                           suppress_warnings=True, 
                           stepwise=True)
    model.fit(arima_data.Close)
    forecast = model.predict(trend='nc',n_periods=5) #(0,1,0)인수 나오는것 처리해야함
    predictions = list(forecast)
    print(predictions)
    print(code)

    #sample_submission.loc[:,code] = predictions*2
    pred_case.append(predictions)
    sample_submission.loc[:4,code] = predictions
sample_submission.isna().sum().sum()

pred_case=np.array(pred_case)

  0%|          | 0/5 [00:00<?, ?it/s]


ValueError: cannot reshape array of size 265 into shape (2)

In [ ]:
sample_submission.to_csv('BASELINE_Linear_test.csv',index=False)

In [ ]:
sample_submission

,Day,000060,000080,000100,000120,000150,000240,000250,000270,000660,...,330860,336260,336370,347860,348150,348210,352820,357780,363280,950130
0,2021-11-01,0,0,0,0,0,0,0,0,102338.927380,...,0,0,0,0,0,0,0,0,0,0
1,2021-11-02,0,0,0,0,0,0,0,0,102684.832052,...,0,0,0,0,0,0,0,0,0,0
2,2021-11-03,0,0,0,0,0,0,0,0,102025.531475,...,0,0,0,0,0,0,0,0,0,0
3,2021-11-04,0,0,0,0,0,0,0,0,102397.978371,...,0,0,0,0,0,0,0,0,0,0
4,2021-11-05,0,0,0,0,0,0,0,0,101472.182547,...,0,0,0,0,0,0,0,0,0,0
5,2021-11-29,0,0,0,0,0,0,0,0,0.000000,...,0,0,0,0,0,0,0,0,0,0
6,2021-11-30,0,0,0,0,0,0,0,0,0.000000,...,0,0,0,0,0,0,0,0,0,0
7,2021-12-01,0,0,0,0,0,0,0,0,0.000000,...,0,0,0,0,0,0,0,0,0,0
8,2021-12-02,0,0,0,0,0,0,0,0,0.000000,...,0,0,0,0,0,0,0,0,0,0
9,2021-12-03,0,0,0,0,0,0,0,0,0.000000,...,0,0,0,0,0,0,0,0,0,0


In [ ]:

start_date = '20211101'
end_date = '20211105'

start_weekday = pd.to_datetime(start_date).weekday()
max_weeknum = pd.to_datetime(end_date).strftime('%V')
Business_days = pd.DataFrame(pd.date_range(start_date,end_date,freq='B'), columns = ['Date'])

print(f'WEEKDAY of "start_date" : {start_weekday}')
print(f'NUM of WEEKS to "end_date" : {max_weeknum}')
print(f'HOW MANY "Business_days" : {Business_days.shape}', )


real=[]
real_case=[]
real_data=pd.read_csv('C:\\Users\\user\\Desktop\\computing\\outside\\dacon\\stock_predict2\\open\\sample_submission.csv')
for code in tqdm(sample_stock_list['종목코드'].values):
    data = fdr.DataReader(code, start = start_date, end = end_date)[['Close']].reset_index()
    data = pd.merge(Business_days, data, how = 'outer')
    data['weekday'] = data.Date.apply(lambda x : x.weekday())
    data['weeknum'] = data.Date.apply(lambda x : x.strftime('%V'))
    data.Close = data.Close.ffill()
    data.Close = data.Close.bfill()

    real_data_close = pd.pivot_table(data = data, values = 'Close', columns = 'weekday', index = 'weeknum')
    real_data_close_flatten = np.ravel(real_data_close.to_numpy())
    # print(real_data_close_flatten)
    real_case.append(real_data_close_flatten)

    real_data.loc[:4,code]=real_data_close_flatten


type(real_data)
type(sample_submission)

label_np = real_data.to_numpy()[0:5,1:len(sample_stock_list)]
pred_np = sample_submission.to_numpy()[0:5, 1:len(sample_stock_list)]
real_case=np.array(real_case)
pred_case=pred_case.reshape(-1,5)



result=np.mean(abs(real_case-pred_case)/real_case)*100

print(result)

WEEKDAY of "start_date" : 0
NUM of WEEKS to "end_date" : 44
HOW MANY "Business_days" : (5, 1)


100%|██████████| 5/5 [00:01<00:00,  3.47it/s]

1.4705716142802925
